In [72]:
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
import pandas as pd

let's try estimating sin(x)

In [6]:
x_set = np.linspace(0,1, 1000)

In [8]:
y_set = np.sin(x_set)

In [4]:
class Network(nn.Module):
    
    def __init__(self, input_size, output_size):
        #make a 2 HL 64 neuron network, fully connected, and test its power
        super(Network, self).__init__()
        
        self.pi = nn.Sequential(nn.Linear(input_size, 64),
                                nn.ReLU(),
                                nn.Linear(64, 64),
                                nn.ReLU(),
                                nn.Linear(64, output_size))
        
    def forward(self, x):
        return self.pi(x)

In [9]:
testnet = Network(1,1)

In [11]:
#learnable parameters
list(testnet.parameters())

[Parameter containing:
 tensor([[ 7.4210e-01],
         [-6.2422e-01],
         [ 7.0136e-02],
         [ 6.9494e-01],
         [-1.3752e-01],
         [-7.6551e-01],
         [ 3.0112e-01],
         [ 4.3266e-02],
         [ 7.8482e-01],
         [-7.1765e-01],
         [-3.2632e-01],
         [-1.1904e-01],
         [-2.9981e-01],
         [-7.8621e-01],
         [-9.1743e-04],
         [-8.1548e-01],
         [ 3.4327e-01],
         [ 7.8124e-01],
         [-7.9956e-01],
         [ 5.8061e-01],
         [ 8.3889e-03],
         [ 2.4761e-02],
         [ 2.6582e-01],
         [ 4.6382e-01],
         [ 8.5130e-01],
         [ 7.2295e-01],
         [ 5.5010e-01],
         [-3.8199e-01],
         [-3.0772e-01],
         [ 1.2265e-01],
         [ 5.2200e-01],
         [ 5.2562e-01],
         [ 7.2379e-01],
         [ 9.0143e-01],
         [ 5.6913e-01],
         [ 4.6732e-01],
         [-7.7757e-01],
         [-3.2066e-01],
         [-7.5434e-01],
         [-9.6266e-01],
         [ 4.8659

In [44]:
inputx

tensor([0.], dtype=torch.float64)

In [48]:
yin = np.array([y_set[0]])

In [42]:
xin = np.array([x_set[0]])

In [43]:
inputx = torch.from_numpy(xin)

In [49]:
inputy = torch.from_numpy(yin)

In [45]:
testnet = testnet.double()

In [51]:
out = testnet(inputx.double())

In [57]:
#zero the grad so we start over, or else grad will compound
testnet.zero_grad()

In [61]:
criterion = nn.MSELoss()

In [62]:
loss = criterion(out.double(), inputy.double())

In [64]:
loss

tensor(0.0020, dtype=torch.float64, grad_fn=<MseLossBackward>)

In [65]:
#backprop
loss.backward()

In [66]:
#easy SGD
#new_param = old_param + (grad*lr)
lr = 1e-3

for param in testnet.parameters():
    #f.data.sub_(f.grad.data * learning_rate)
    param.data.sub_(param.grad.data*lr)

In [68]:
#instead, use optimizers form torch.optim
optimizer = optim.SGD(testnet.parameters(), lr=1e-3)

In [ ]:
#in training:
optimizer.zero_grad()
out = testnet(inputx.double())
loss = criterion(out.double(), inputy.double())
loss.backward() #calc backwards
optimizer.step() #do the actual update


# actual training

In [203]:
#make dataset
x = np.linspace(0,6,10000)
y = np.sin(x)

In [204]:
dataset = list(zip(x, y))

In [205]:
len(dataset)

10000

In [206]:
dataset[0]

(0.0, 0.0)

In [210]:
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [211]:
criterion = nn.MSELoss()

In [249]:
training_epochs = 100
optimizer = optim.SGD(testnet.parameters(), lr=1e-6)

In [250]:
for epoch in range(training_epochs):
    testnet.train()
    
    for xval, yval in dataloader:
        #unsqueeze just puts in in row form
        yval = yval.double()
        xval = xval.double()

        optimizer.zero_grad()
        out = testnet(xval.unsqueeze(1))
        loss = criterion(out, yval.unsqueeze(1))
        loss.backward() #calc backwards
        optimizer.step() #do update
        
    #print some stats
    if epoch%10 ==0:
        with torch.no_grad():
            testxval = torch.from_numpy(np.random.rand(1)*6)
            testyval = torch.from_numpy(np.array([np.sin(testxval.item())]))

            pred = testnet(testxval.double())
            testloss = criterion(pred.double(), testyval.double())

            print(f'curr MSE loss on test: {round(testloss.item(),4)}')
    
    

curr MSE loss on test: 0.0001
curr MSE loss on test: 0.0001
curr MSE loss on test: 0.0
curr MSE loss on test: 0.0004
curr MSE loss on test: 0.0
curr MSE loss on test: 0.0001
curr MSE loss on test: 0.0
curr MSE loss on test: 0.003
curr MSE loss on test: 0.0
curr MSE loss on test: 0.0


In [251]:
x = np.random.rand(1)*6

In [252]:
np.sin(x)

array([0.98194848])

In [253]:
testnet.eval()

Network(
  (pi): Sequential(
    (0): Linear(in_features=1, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [254]:
testnet(torch.tensor([x]).double())

tensor([[0.9796]], dtype=torch.float64, grad_fn=<AddmmBackward>)